
---

# 🧬 Generic Models

### 🎯 Intent

Create **reusable, type-safe containers** (like `Page[User]`, `ApiResponse[int]`) using Python generics in Pydantic v2.

---

### 🧩 Core Components

1. **🧪 Define a Generic**

   * Use `TypeVar` + `Generic[T]`.
   * Example:

   ```python
   T = TypeVar("T")

   class Box(BaseModel, Generic[T]):
       item: T
   ```

2. **📦 Use with Concrete Types**

   * Instantiate with real types:

     * `Box[int](item=1)`
     * `Box[str](item="hi")`

3. **🔁 Nested Generics**

   * Build wrappers like:

   ```python
   class Page(BaseModel, Generic[T]):
       items: list[T]
       total: int
   ```

   * `Page[User]` → validates a list of `User`.

4. **📤 Serialization & Schema**

   * `.model_dump()` works normally.
   * `.model_json_schema()` shows correct type (e.g., `items` of `Page[User]` → `User` schema).

5. **🧰 When to Use**

   * Response wrappers (`ApiResponse[T]`)
   * Pagination (`Page[T]`)
   * Generic containers (e.g., `Pair[K, V]`)

---


In [1]:
from typing import TypeVar, Generic
from pydantic import BaseModel
from pydantic.generics import GenericModel

# 👇 Define a type variable
T = TypeVar("T")

# 1️⃣ Simple Generic Box
class Box(GenericModel, Generic[T]):
    item: T

# 2️⃣ Pagination Wrapper
class Page(GenericModel, Generic[T]):
    items: list[T]
    total: int

# 3️⃣ API Response Wrapper
class ApiResponse(GenericModel, Generic[T]):
    data: T
    error: str | None = None


# ✅ Example Usage
class User(BaseModel):
    id: int
    name: str

if __name__ == "__main__":
    # Box[int]
    b = Box[int](item=123)
    print("Box[int] →", b.model_dump())

    # Page[User]
    page = Page[User](items=[{"id": 1, "name": "Mukesh"}], total=1)
    print("Page[User] →", page.model_dump())

    # ApiResponse[User]
    resp = ApiResponse[User](data={"id": 42, "name": "Alice"})
    print("ApiResponse[User] →", resp.model_dump())

    # Schema Generation
    print("Page[User] Schema →", Page[User].model_json_schema())


Box[int] → {'item': 123}
Page[User] → {'items': [{'id': 1, 'name': 'Mukesh'}], 'total': 1}
ApiResponse[User] → {'data': {'id': 42, 'name': 'Alice'}, 'error': None}
Page[User] Schema → {'$defs': {'User': {'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['id', 'name'], 'title': 'User', 'type': 'object'}}, 'properties': {'items': {'items': {'$ref': '#/$defs/User'}, 'title': 'Items', 'type': 'array'}, 'total': {'title': 'Total', 'type': 'integer'}}, 'required': ['items', 'total'], 'title': 'Page[User]', 'type': 'object'}


/opt/anaconda3/envs/genai-doc-chat/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.generics:GenericModel` has been moved to `pydantic.BaseModel`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
